In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
autentica = pd.read_excel('ls.xlsx')
login = str(autentica['login'][0])
senha = str(autentica['senha'][0])
nome = 'CAFI'

display(autentica)

,login,senha
0,2184579,sis1986%
1,1966990,19891150
2,2184708,andela@78


In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By


from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import time

def abrir_suap():
    
    global nav
    nav = webdriver.Chrome()
    nav.get("https://suap.ifpe.edu.br/accounts/login/?next=/")
    # inserindo usuário
    nav.find_element(By.XPATH, '//*[@id="id_username"]').click()
    nav.find_element(By.XPATH, '//*[@id="id_username"]').send_keys(login)

    # inserindo senha 
    nav.find_element(By.XPATH, '//*[@id="id_password"]').click()
    nav.find_element(By.XPATH, '//*[@id="id_password"]').send_keys(senha)

    nav.find_element(By.XPATH, '//*[@id="ok"]').click()

In [5]:
# extraindo para lista
def table_to_list():
#     abrir_suap()
    tabela = []
    
    try:
        pag = 1
        saida = False
        while saida is False and pag < 3:
            
            nav.get(f'https://suap.ifpe.edu.br/patrimonio/inventario_busca/?page={pag}&status=&empenho=&empenho=&data_final=&sala=&numero_serie=&numero=&rotulos=&data_inicial=&estado_conservacao=6&elemento_despesa=&valor=&responsavel=&responsavel=&contabil=&fornecedor=&fornecedor=&setor_responsavel=&campus=12&descricao=')

    #         condição de parada quando não houver mais a classe row1, pq a tabela ainda é montada, mas sem linhas
            celula = nav.find_element(By.CLASS_NAME, 'row1')

            if celula:

    #             Acha a tabela e todos os elementos td
                lista = nav.find_element(By.TAG_NAME, 'table')
                td = lista.find_elements(By.TAG_NAME, 'td')
            
                for i in td:
                    tabela.append(i.text)
                pag += 1

            else:
                saida = True

        return tabela
    
    except NoSuchElementException:
        print('acabaram as páginas')
        
table = table_to_list()

In [7]:
display(table)

[' ',
 '',
 '33\nAtivo',
 '449052-42',
 '182CREC - BANCO P/DESENHO (FIXO COM 75CM DE ALTURA ASSENTO REDONDO REF 42102)',
 'VANESSA RAMOS ALVES (CAFI AFI-DEN)',
 'RECIFE',
 '',
 '',
 '16/06/2010',
 '23/12/2010',
 'IFPE - CAMPUS RECIFE (10.767.239/0003-07)',
 '0,01',
 '',
 '3842\nAtivo',
 '449052-42',
 '14471CREC - ARMARIO (DE A O PARA ACESSORIOS DE MAQUINAS)',
 'Carlos Eduardo Cabral Rodrigues (CAFI AFI-CCBEC)',
 'RECIFE',
 '',
 '',
 '16/06/2010',
 '23/12/2010',
 'IFPE - CAMPUS RECIFE (10.767.239/0003-07)',
 '0,01',
 '',
 '16809\nAtivo',
 '449052-10',
 '51132CREC - TABELA DE BASQUETEBOL MOVEL (PAR) MOD.VITA, EM VIDRO TEMP.10MM, MED.1,80 X 1,05 C/AROS, MARCA RAPINI',
 'Ezenildo Emanuel de Lima (CAFI AFI-CODAGR)',
 'AFOGADOS DA INGAZEIRA',
 '',
 '',
 '16/06/2010',
 '23/12/2010',
 'IFPE - CAMPUS RECIFE (10.767.239/0003-07)',
 '12.698,00',
 '',
 '17274\nAtivo',
 '449052-35',
 '51615CREC - MICROCOMPUTADOR PESSOAL (NOTEBOOK INTEL 1.7 GHZ, MARCA DELL.)',
 'Ezenildo Emanuel de Lima (CAFI AFI-CO

In [ ]:

# nav.get('https://suap.ifpe.edu.br/patrimonio/inventario_busca/?page=1&status=&empenho=&empenho=&data_final=&sala=&numero_serie=&numero=&rotulos=&data_inicial=&estado_conservacao=6&elemento_despesa=&valor=&responsavel=&responsavel=&contabil=&fornecedor=&fornecedor=&setor_responsavel=&campus=12&descricao=')
# nav.get('https://suap.ifpe.edu.br/patrimonio/inventario_busca/?page=1&status=&empenho=&empenho=&data_final=&sala=&numero_serie=&numero=33-373551&rotulos=&data_inicial=&estado_conservacao=6&elemento_despesa=&valor=&responsavel=&responsavel=&contabil=&fornecedor=&fornecedor=&setor_responsavel=&campus=12&descricao=')
def build_df():
    df_tabela = pd.DataFrame()


    tombos = []
    ele_des = []
    descricao = []
    cargas = []
    c_contabil = []
    salas = []
    dt_entrada = []
    valores = []

    #  remove o primeiro elemento ta tabela, informação quebrada    
    for excluir in range(0, len(tabela), 1201):
        tabela.pop(excluir)

    for tombo in range(1, len(tabela)+1, 12):
        if tabela[tombo] == '':
            tabela.pop(tombo)

    # listas para montar o data frame
    for tombo in range(1, len(tabela), 12):
        tombos.append(tabela[tombo])

    for ed in range(2, len(tabela), 12):
        ele_des.append(tabela[ed])

    for des in range(3, len(tabela), 12):
        descricao.append(tabela[des])

    for carga in range(4, len(tabela), 12):
        cargas.append(tabela[carga])

    for c in range(5, len(tabela), 12):
        c_contabil.append(tabela[c])

    for sala in range(6, len(tabela), 12):
        salas.append(tabela[sala])

    for data in range(8, len(tabela), 12):
        dt_entrada.append(tabela[data])

    for valor in range(11, len(tabela), 12):
        valores.append(tabela[valor])



    #  montagem do data frame
    df_tabela['tombo'] = tombos
    df_tabela['tombo'] = df_tabela['tombo'].astype(str).str[:-6]

    df_tabela['ed'] = ele_des

    df_tabela['descricao'] = descricao

    df_tabela['carga'] = cargas

    df_tabela['c_contabil'] = c_contabil

    df_tabela['dt_entrada'] = dt_entrada
    #     df_tabela['dt_entrada'] = pd.to_datetime(df_tabela['dt_entrada'])

    df_tabela['valor'] = valores
    #     df_tabela['valor'] = df_tabela['valor'].str.replace('.','',regex=True)
    #     df_tabela['valor'] = df_tabela['valor'].str.replace(',','.',regex=True)
    #     df_tabela['valor'] = pd.to_numeric(df_tabela['valor'])
    #     df_tabela['valor'] = df_tabela['valor'].round(2)


    #     df_tabela.to_excel(f'suap/{nome}_relatorio_patrimonio.xlsx', index=False)

    display(df_tabela)

In [ ]:
# montar_base()
listav = []
# print(tabela)
df_tabela.to_clipboard()
for tombo in range(2, len(tabela), 12):
    if tabela[tombo] == '':
        listav.append(tabela[tombo])
        print(f'posicao - {tombo}  -  espaço vazio')

print(len(listav))   

In [ ]:
for tombo in range(2, len(tabela), 12):
        listav.clear()
        listav.append(tabela[tombo])
        print(f'posicao - {tombo}  -  valor {tabela[tombo]}') 